# Convertible Bond Valuation

In [1]:
import sys
sys.path.append("..")
sys.path.append("..\\..")

In [2]:
from importlib import reload

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from financepy.products.bonds.FinConvertibleBond import FinConvertibleBond
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinCalendar import FinCalendarTypes
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinCalendar import FinDayAdjustTypes, FinDateGenRuleTypes
from financepy.market.curves.FinFlatCurve import FinFlatCurve
from financepy.finutils.FinSchedule import FinSchedule

## Creating the Convertible Bond

In [5]:
startConvertDate = FinDate(20, 12, 2019)
maturityDate = FinDate(20, 3, 2023)
face = 100.0 
conversionRatio = 5.0  # Number of shares for face amount
coupon = 0.0575
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualBasis = FinDayCountTypes.ACT_365_FIXED

## Valuation

### No Calls and Puts

In [6]:
settlementDate = FinDate(31, 12, 2019)

In [7]:
dividendSchedule = FinSchedule(settlementDate, maturityDate).flows()[1:]
dividendYields = [0.01] * len(dividendSchedule)

In [30]:
stockPrice = 20
stockVolatility = 0.30
rate = 0.0375
discountCurve = FinFlatCurve(settlementDate, rate, 1)
creditSpread = 0.02
recoveryRate = 0.5
numStepsPerYear = 100

In [31]:
callPrice = 10000 # Make call uneconomic so it does not impact pricing
callDates = [FinDate(20, 3, 2020), FinDate(20, 3, 2021), FinDate(20, 3, 2022)]
callPrices = [callPrice, callPrice, callPrice]

In [32]:
putPrice = 0.0 # Make call uneconomic so it does not impact pricing
putDates = [FinDate(20, 3, 2020), FinDate(20, 3, 2021), FinDate(20, 3, 2022)]
putPrices = [putPrice, putPrice, putPrice]

In [33]:
bond = FinConvertibleBond(maturityDate, coupon, frequencyType, startConvertDate, conversionRatio,
                          callDates, callPrices, putDates, putPrices, accrualBasis, face)

Peform the valuation

In [34]:
bond.value(settlementDate, stockPrice, stockVolatility, dividendSchedule, dividendYields,
           discountCurve, creditSpread, recoveryRate, numStepsPerYear)

{'cbprice': 125.40612491584183,
 'bond': 101.49138121192846,
 'delta': 3.6311762193117416,
 'gamma': 2.736764837048754,
 'theta': 170.17965629393188}

'cbrice' is the bond price. 'bond' is the price of the bond alone without any conversion option

## Impact of Calls

In [42]:
callPrice = 110.0
callPrices = [callPrice, callPrice, callPrice]

Need to recreate the convertible bond.

In [43]:
bond = FinConvertibleBond(maturityDate, coupon, frequencyType, startConvertDate, conversionRatio,
                          callDates, callPrices, putDates, putPrices, accrualBasis, face)

In [44]:
bond.value(settlementDate, stockPrice, stockVolatility, dividendSchedule, dividendYields,
           discountCurve, creditSpread, recoveryRate, numStepsPerYear)

{'cbprice': 109.92013214243772,
 'bond': 101.49138121192846,
 'delta': 1.2259713224677258,
 'gamma': 4.247967725642335,
 'theta': 67.3924135278653}

## Stock Price Sensitivity

As the stock price goes up, the conversion options becomes more valuable

In [45]:
stockPrices = np.linspace(0,40,11)
print("%9s %12s %12s %12s"% ("StockPx", "CB Price", "Bond Price", "Delta"))
for stockPrice in stockPrices:
    res = bond.value(settlementDate, stockPrice, stockVolatility, dividendSchedule, dividendYields,
                     discountCurve, creditSpread, recoveryRate, numStepsPerYear)
    print("%9.2f %12.4f %12.4f %12.4f"% (stockPrice, res['cbprice'], res['bond'], res['delta']))

  StockPx     CB Price   Bond Price        Delta
     0.00     101.4914     101.4914       0.0000
     4.00     101.4921     101.4914       0.0017
     8.00     101.7381     101.4914       0.1978
    12.00     103.5986     101.4914       0.7412
    16.00     107.1775     101.4914       0.7920
    20.00     109.9201     101.4914       1.2260
    24.00     121.3883     101.4914       4.2749
    28.00     140.4348     101.4914       4.9797
    32.00     160.4156     101.4914       4.9999
    36.00     180.4155     101.4914       5.0000
    40.00     200.4155     101.4914       5.0000


As expected the price rises with increasing stock price as the conversion option is more valuable

## Convergence Testing

In [46]:
stockPrice = 22.0
stockVolatility = 0.30

print("%9s %12s %12s %12s"% ("Steps/Yr", "CB Price", "Bond Price", "Delta"))
for numStepsPerYear in [5,10,20,30,40,50,60,70,80,90,100,200,500]:
    res = bond.value(settlementDate, stockPrice, stockVolatility, dividendSchedule, dividendYields,
                     discountCurve, creditSpread, recoveryRate, numStepsPerYear)
    print("%9.2f %12.4f %12.4f %12.4f"% (numStepsPerYear, res['cbprice'], res['bond'], res['delta']))

 Steps/Yr     CB Price   Bond Price        Delta
     5.00     110.0000     101.4730       0.0000
    10.00     113.2999     101.4823       2.7891
    20.00     113.6008     101.4873       2.8905
    30.00     113.6987     101.4890       2.9253
    40.00     113.7638     101.4898       2.9194
    50.00     113.7962     101.4903       2.9182
    60.00     113.8204     101.4907       2.9147
    70.00     113.9720     101.4909       2.9185
    80.00     113.9715     101.4911       2.9175
    90.00     114.0750     101.4913       2.9199
   100.00     114.0649     101.4914       2.9185
   200.00     114.0976     101.4919       2.9230
   500.00     114.0999     101.4922       2.9253
